# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №5 - Алгоритмы кластеризации


###### <hr\>
**Общая информация**

**Срок сдачи:** 17 декабря 2018, 06:00 <br\>
**Штраф за опоздание:** -2 балла после 06:00 17 декабря, -4 балла после 06:00 24 декабря, -6 баллов после 06:00 31 декабря -8 баллов  после 06:00 7 января


Используйте данный Ipython Notebook при оформлении домашнего задания.
<hr\>

**Из чего состоит задание:**

###### Теоретические вопросы (2 балла)

###### Реализация алгоритма кластеризации (6 баллов)

###### Боевое применение (2 балла)




**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw4.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

# Определяем, что вам досталось

Укажите свою фамилию на русском языке в поле ниже.

In [215]:
import hashlib

def lucky_hash(text):
    return int(hashlib.md5(text.encode('utf-8')).hexdigest()[:8], 16) 

USER_NAME = u"Иванов".lower()

ALGORITHMS = [
    u"Gaussian Mixture Model с использованием maximum a-posteriori для выбора кластера (параметр - число кластеров)",
    u"Hierarchical clustering с поддержкой single-linkage, complete-linkage, average-linkage (параметры - число кластеров, linkage)",]

print ("Имплементируйте алгоритм кластеризации %s"\
% (
    ALGORITHMS[lucky_hash(USER_NAME[::-1]) % 2]
) )

Имплементируйте алгоритм кластеризации Hierarchical clustering с поддержкой single-linkage, complete-linkage, average-linkage (параметры - число кластеров, linkage)


Не стоит переживать из-за Вашего варианта. Технически оба алгоритма несложно реализовать.  EM алгоритма бояться не стоит, Вам будет нужно просто реализовать уже выведенные формулы для E и M шагов.

In [349]:
import time
import pandas as pd
import pylab as pl
import numpy as np
import scipy.spatial as ss
import sklearn.cluster as sc
import sklearn.manifold as sm
import sklearn.datasets as ds
import sklearn.metrics as smt
from sklearn.mixture import GaussianMixture
from sklearn.cluster import AgglomerativeClustering

import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)

# Plotting config
%pylab inline

%load_ext pycodestyle_magic

Populating the interactive namespace from numpy and matplotlib
The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


# Теоретическое введение


Основная задача кластерного анализа — разбиение исходного набора объектов на группы (кластеры) таким образом, чтобы объекты в группе были похожи друг на друга, а объекты из разных групп - отличались. 

В этой работе мы будем реализовывать один из двух популярных методов кластеризации: разделение смеси нормальных распределений или агломеративную кластеризацию. 

## Смесь нормальных распределений



Предполагаем, что наши данные порождены смесью $k$ нормальных распределений, то есть 

$$ p(\mathbf{x}) = \sum_k \pi_k \mathcal{N}(\mathbf{x} | \mathbf{\mu}_k, \mathbf{\Sigma}_k) $$,


где $\mathcal{N}$ - многомерное нормальное распределение размерности ${D}$:

$$\mathcal{N(\mathbf{x} | \mathbf{\mu}, \mathbf{\Sigma}}) = \frac{1}{(2 \pi)^{D/2}} \frac{1}{|\mathbf{\Sigma}|^{1/2}} \exp \left\{-\frac{1}{2}(\mathbf{x} - \mathbf{\mu})^T \mathbf{\Sigma^{-1}} (\mathbf{x} - \mathbf{\mu})\right\}$$

**Параметры**  


${D}$-мерный вектор средних

$$\mathbf{\mu}$$

$D \times D$-мерная матрица ковариации   (симметричная)


$$\mathbf{\Sigma} = E[(\mathbf{x} - \mathbf{\mu})(\mathbf{x} - \mathbf{\mu})^T]$$



Необходимо оценить параметры $\pi_k, \mathbf{\mu_k}, \mathbf{\Sigma_k} $ для всех компонент смеси.  Сделать это можно с помощью EM алгоритма - алгоритма, который находит оценку максимального правдоподобия в задаче со скрытыми переменными. 

## Expectation Maximization

**I** До цикла проинциализировать случайно  $\mu_k, \Sigma_k, \pi_k$ ($\pi_k$ должны суммироваться в 1)

**E** Expectation: при фиксированных $\mu_k, \Sigma_k, \pi_k$
$$
p(z_k = 1| x_n ) = \gamma(z_{nk}) = \frac{\pi_k \mathcal{N} (\mathbf{x}_n | \mu_k, \Sigma_k)}{\sum_{j=1}^K \pi_j \mathcal{N} (\mathbf{x}_n | \mu_j, \Sigma_j)}
$$
**M** Maximization: при фиксированных $\gamma(z_{nk})$
$$
N_k = \sum_{n=1}^N \gamma(z_{nk}), \;\; \mu_k = \frac 1 {N_k} \sum_{n=1}^N \gamma(z_{nk}) \mathbf{x}_n
$$
$$
\Sigma_k = \frac 1 {N_k} \sum_{n=1}^N \gamma(z_{nk}) (\mathbf{x}_n - \mu_k)(\mathbf{x}_n - \mu_k)^T
$$
$$
\pi_k = \frac{N_k}{N}
$$
**S** Остановиться при достижении сходимости

Номер кластера для $n$ объекта будем брать по принципу maximum a-posteriori, то есть относить объект к тому кластеру, у которого максимальна апостериорная вероятность:
$$ k_n =  \arg\underset{k}\max  p(z_k = 1| x_n ) =  \arg\underset{k}\max \gamma(z_{nk}) $$

## Агломеративная кластеризация

Иерархические алгоритмы кластеризации строят на выборке систему вложенных разбиений.  Наиболее часто испоьзуемые иерархические алгоритмы - агломеративные алгоритмы, которые строят ирархию по восходящей, то есть от момента, когда все элементы являются отдельными кластерами, до того момента, пока вся выборка не станет одним кластером.

Сам алгоритм описать можно примерно так:

* начинаем с ситуации, когда каждый объект - отдельный кластер
* на каждом шаге совмещаем два наиболее близких кластера
* останавливаемся, когда получаем требуемое количество или единственный кластер


В данной работе Вам предлагается реализовать три варианта расстояний между кластерами, по которым Вы определяете пару для слияния: 
* Single linkage
$$ d_{min}(C_i, C_j) = \min_{\mathbf{x} \in C_i, \mathbf{x}' \in C_j} \|\mathbf{x} -\mathbf{x}' \| $$

* Complete linkage
$$ d_{max}(C_i, C_j) = \max_{\mathbf{x} \in C_i, \mathbf{x}' \in C_j} \|\mathbf{x} -\mathbf{x}' \| $$

* Average linkage
$$ d_{avg}(C_i, C_j) = \frac{1}{n_i n_j}\sum_{\mathbf{x} \in C_i}\sum_{\mathbf{x}' \in C_j} \|\mathbf{x} -\mathbf{x}' \| $$

Для простоты будем использовать евклидово расстояние между объектами.



Для того, чтобы на втором этапе нам не пересчитывать заново расстояния между всеми парами кластеров, можно использовать Формулы Ланса-Вильямса, которые позволяют посчитать расстояние между кластерами после объединения, используя только расстояния между кластерами до объедения.  Таким образом, достаточно будет посчитать расстояния только между всеми одноэлементными кластерами (первый этап), а затем пересчитывать расстояния при их объединении через 
формулы Ланса-Вильямса. 

Общая  формула выглядит так:
$$ d(C_i \cup C_j, C_k) = a_i \cdot d(C_i, C_k) + a_j \cdot d(C_j, C_k) + b \cdot d(C_i, C_j) + c \cdot |d(C_i, C_k) - d(C_j, C_k)|$$

Можно показать, что все относительно разумные кластерные расстояния можно описать данной формулой. 

Для интересущих нас linkage:

* single-linkage       $a_i = \frac{1}{2}, a_j =  \frac{1}{2}, b = 0, c =  - \frac{1}{2} $

* complete-linkage     $a_i = \frac{1}{2}, a_j =  \frac{1}{2}, b = 0, c =   \frac{1}{2} $

* average-linkage       $a_i = \frac{|C_i|}{|C_i \cup C_j|}, a_j =  \frac{|C_j|}{|C_i \cup C_j|}, b = 0, c = 0 $



# Теоретические вопросы (2 балла)
В этой части Вам будут предложены теоретичские вопросы и задачи по теме. Вы, конечно, можете списать их у своего товарища или найти решение в интернете, но учтите, что они обязательно войдут в теоретический коллоквиум. Лучше разобраться в теме сейчас и успешно ответить на коллоквиуме, чем списать, не разобравшись в материале, и быть терзаемым совестью. 


Формулы надо оформлять в формате **LaTeX**.

Задача 1. Сходимость kmeans.

Метод kmeans является частным случаем EM алгоритма и наиболее простым в обосновании сходимости. Докажите, что kmeans всегда сходится, и итераций не будет превышать $k^N$, где $k$ $-$ число кластеров, $N$ $-$ число объектов

Подсказка: Вам стоит рассмотреть функционал, который минимизирует алгоритм, и подумать, как он изменяется на E шаге и на M шаге.



***Решение***

Рассмотрим функционал $$L(C) = \sum_k \sum_{i \in C_k} ||x_i - \mu_k||^2$$

E шаг:

При фиксированных $C_k$ функционал достигает минимума при $\mu_k$ равном среднему (решение задачи МНК в в скалярном случае).

M шаг:

Каждый $x_i$ переопределяется к классу, центроид которого ближе всего к нему. Значит, функционал не возрастает.

Число разбиений $N$ точек на $k$ кластеров равно $k^N$. Если на какой-то итерации $L(C^{(i-1)})=L(C^{(i)})$, центроиды не обновятся и алгоритм остановится. Такое состояние будет гарантировано достигнуто за $k^N$ итераций.

Задача 2. Сходимость EM.

Разобравшись со сходимостью kmeans, можно догадаться и до ответа про EM алгоритм.

При каком условии на правдоподобие системы EM алгоритм будет  сходиться?



***Решение***

На каждой итерации EM-алгоритма полное правдоподобие системы не убывает.

Поэтому, если правдоподобие ограничено сверху, то алгоритм сойдется (куда-нибудь).

Задача 3. Формулы Ланса-Вильямса.

Формулы Ланса-Вильямса крайне удобны для быстрого пересчта расстояний в агломеративной кластеризации. Давайте докажем  формулы для single и complete linkage. 

Докажите, что:

$d_{\min}(U \cup V, S) = \frac{1}{2} d_{\min}(U, S) + \frac{1}{2} d_{\min}(V, S)  - \frac{1}{2} | d_{\min}(U, S) - d_{\min}(V, S) |  $


$d_{\max}(U \cup V, S) = \frac{1}{2} d_{\max}(U, S) + \frac{1}{2} d_{\max}(V, S)  + \frac{1}{2} | d_{\max}(U, S) - d_{\max}(V, S) |  $




***Решение***

Single linkage:

$d_{min}(A, B) = min_{a \in A, b \in B}||a-b||$

$d_{min}(U \cup V, S) = min(d_{min}(U, S), d_{min}(V, S))$

Пусть min достигается на $x \in U, s \in S$.

Тогда $d_{min}(U \cup V, S) = d_{min}(U, S)$ и $d_{min}(U, S) <= d_{min}(V, S)$

$|d_{min}(U, S)-d_{min}(V, S)| = d_{min}(U, S) - d_{min}(V, S)$

И $d_{min}(U \cup V, S) = d_{min}(U, S)$.

Аналогично, если  $x \in V, s \in S$.

Те же приемы для complete linkage.

# Реализация алгоритма кластеризации  (6 баллов)

Алгоритм кластеризации должен удовлетворять следующему интерфейсу. Конструктор принимает набор параметров, необходимых для работы алгоритма кластеризации. Метод `fit` подсчитывает параметры модели и возвращает `self`. Метод `predict` возвращает вектор с индексами кластеров для поданных в него объектов `x`. 

In [356]:
#%%pycodestyle

from sklearn.base import BaseEstimator, ClusterMixin
from sklearn.metrics.pairwise import euclidean_distances as dist


class Clustering(BaseEstimator, ClusterMixin):
    """
    Implement clustering algorithm according
    """

    def __init__(self, k=2, algorithm='single'):
        """
        Please add necessary algoritm parameters to class constructor.
        """
        self.k = k
        self.algorithm = algorithm
        if self.algorithm == 'single':
            self.linkage = self.single_linkage
        elif self.algorithm == 'complete':
            self.linkage = self.complete_linkage
        else:
            print('Unknown linkage type')
        return

    def single_linkage(self, i, j):
        d_i = self.dist_matrix[i]
        d_j = self.dist_matrix[j]
        self.dist_matrix[i] = 0.5 * d_i + 0.5 * d_j - 0.5 * np.abs(d_i - d_j)
        # Теперь в матрице расстояний новому кластеру
        # соответствует i строка и столбец.
        # Inf запрещает обращаться к столбцу и строке j
        # (они больше не соответвуют никакому кластеру)
        self.dist_matrix[j] = np.inf
        self.dist_matrix[:, j] = np.inf
        # Если возникла неопределенность в новом столбце
        # (из-за арифметики с Inf),
        # принудительно ставим Inf,
        # ведь это столбец/строка по построению соответствуют
        # удаленным кластерам или расстоянию объекта до себя.
        for elem in range(self.N):
            if np.isnan(self.dist_matrix[i, elem]):
                self.dist_matrix[i, elem] = np.inf
        self.dist_matrix[:, i] = self.dist_matrix[i]

    def complete_linkage(self, i, j):
        d_i = self.dist_matrix[i]
        d_j = self.dist_matrix[j]
        self.dist_matrix[i] = 0.5 * d_i + 0.5 * d_j + 0.5 * np.abs(d_i - d_j)
        self.dist_matrix[j] = np.inf
        self.dist_matrix[:, j] = np.inf
        for elem in range(self.N):
            if np.isnan(self.dist_matrix[i, elem]):
                self.dist_matrix[i, elem] = np.inf
        self.dist_matrix[:, i] = self.dist_matrix[i]

    def fit(self, X, y=None):
        """
        Use data matrix x to compute model parameters
        """
        self.N = len(X)
        # Список, элементы которого - номера объектов в кластере.
        self.clusters = [[x] for x in range(self.N)]
        self.dist_matrix = dist(X)
        # Inf гарантирует нам, что не возьмем расстояния объекта до себя.
        np.fill_diagonal(self.dist_matrix, np.inf)
        return self

    def predict(self, x):
        """
        Using computed model parameters predict cluster
        for all objects from x
        """
        iter = 1
        while iter <= self.N - self.k:
            ind = np.argmin(self.dist_matrix)
            i = ind // self.dist_matrix.shape[0]
            j = ind % self.dist_matrix.shape[0]
            if i > j:
                tmp = i
                i = j
                j = tmp
            iter += 1
            # Список, элементы которого - номера объектов в кластере.
            # None - значит кластер пустой.
            # Не удаляю кластеры, чтобы не ломать нумерцию объектов
            # (матрица расстояний не меняет размер)
            self.clusters[i] += self.clusters[j]
            self.clusters[j] = None
            self.linkage(i, j)
        # Убираю пустые кластеры
        for l in range(self.N - self.k):
            self.clusters.remove(None)
        y_pred = np.zeros([x.shape[0]])
        for C in range(len(self.clusters)):
            for x in self.clusters[C]:
                y_pred[x] = C
        return y_pred.astype(int)

    def fit_predict(self, X, y=None):
        self.fit(X, y)
        return self.predict(X)

Сначала проверим реализованный алгоритм кластеризации на классическом наборе данных [Iris](http://www.wikiwand.com/en/Iris_flower_data_set). Загрузим данные (они включены в библиотеку sklearn) и посмотрим на то, как они выглядят в двух проекциях (для простоты используем 2 класса из 3).

In [358]:
#%%pycodestyle

iris = ds.load_iris()
x_iris = iris.data[:100]
y_iris = iris.target[:100]

pl.figure(figsize=(10, 5))

pl.subplot(1, 2, 1)
pl.scatter(x_iris[:, 0], x_iris[:, 1], c=y_iris, cmap=pl.cm.PuOr, lw=0, s=30)
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')

pl.subplot(1, 2, 2)
pl.scatter(x_iris[:, 2], x_iris[:, 3], c=y_iris, cmap=pl.cm.PuOr, lw=0, s=30)
plt.xlabel('Petal length')
plt.ylabel('Petal width')
pl.show()

Видно, что классы разделяются, поэтому можно надеяться, что наш алгоритм "найдет" кластеры, соответствующие исходным классам. Результат работы реализованного алгоритма кластеризации предлагается сравнить с эталонной кластеризацией. Для этого предлагается изучить метрику ([adjusted rand score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_rand_score.html)). В случае если значение этой метрики отличается от 1, предлагается поработать над улучшением реализации своего алгоритма.

In [363]:
#%%pycodestyle

my_clust = Clustering()
start = time.time()
pred_iris = my_clust.fit_predict(x_iris)
timer = time.time() - start
print("Adjusted Rand index for iris is: %.2f"
      % smt.adjusted_rand_score(y_iris, pred_iris))
print(f"My Clustering time is: {timer}")

Adjusted Rand index for iris is: 1.00
My Clustering time is: 0.03454017639160156


/home/semen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in subtract


In [290]:
# Давайте проверим, как справляются sklearn реализации
sklearn_gmm = GaussianMixture(n_components=2)
gmm_t0 = time.time()
sklearn_gmm.fit(x_iris)
gmm_diff = time.time() - gmm_t0
gmm_score = smt.adjusted_rand_score(y_iris, sklearn_gmm.predict(x_iris))

sklearn_aggl = AgglomerativeClustering(n_clusters=2)
aggl_t0 = time.time()
aggl_pred = sklearn_aggl.fit_predict(x_iris)
aggl_diff = time.time() - aggl_t0
aggl_score = smt.adjusted_rand_score(y_iris, aggl_pred)
print ("GMM score {}. GMM time {}. AGGL score {}. AGGL time {}".format(gmm_score, gmm_diff, aggl_score, aggl_diff))

GMM score 1.0. GMM time 0.009765625. AGGL score 1.0. AGGL time 0.0008988380432128906


Сверьте Ваш алгоритм с этими числами. Уверен, что у Вас получится не хуже!

# Боевое применение (2  балла)


Алгоритмы кластеризации прекрасны тем, что они позволяют быстро понять, как устроены наши данные. Давайте применим Ваш алгоритм для анализа реальной задачи!

Важное замечание: в этой части задания студенту самому предлагается разобраться с данными и их форматом в качестве полезного упраженения. Не принимайте это близко к сердцу :)

Ввозьмем датасет различных стран.  
Данные нужно скачать тут https://data.worldbank.org/data-catalog/world-development-indicators.   


Подход следующий: фиксируем год, для каждой страны берём индикаторы по зафиксированному году, кластеризуем по индикаторам. 
Разбиение должно быть объяснимым, в противном случае, скорее всего нужно лучше настроить параметры кластеризации.   

Необходимо показать, какие страны попали в общий кластер в и объяснить из-за каких признаков это произошло. Если страны были в одном кластере, а спустя несколько лет разъехались по разным кластерам, попробуйте это объяснить. 

In [7]:
# Место для Вашего кластерного анализа

In [142]:
df = pd.read_csv('WDIData.csv')
df.head(2)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,Unnamed: 62
0,Arab World,ARB,"2005 PPP conversion factor, GDP (LCU per inter...",PA.NUS.PPP.05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arab World,ARB,"2005 PPP conversion factor, private consumptio...",PA.NUS.PRVT.PP.05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [148]:
df.columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', 'Unnamed: 62'],
      dtype='object')

In [164]:
df.shape

(422400, 63)

In [224]:
df1 = df.loc[:, ['Country Name', 'Indicator Name', '1960']]
df1 = df1.dropna()
df1.shape

(38332, 3)

In [225]:
df2 = df.loc[:, ['Country Name', 'Indicator Name', '2017']]
df2 = df2.dropna()
df2.shape

(130302, 3)

In [226]:
df1 = df1.pivot(index='Country Name', columns='Indicator Name', values='1960')
df1.shape

(263, 403)

In [227]:
df2 = df2.pivot(index='Country Name', columns='Indicator Name', values='2017')
df2.shape

(263, 1014)

In [202]:
df1.fillna(0, inplace=True)
df2.fillna(0, inplace=True)

In [210]:
X1 = df1.values
X2 = df2.values

In [343]:
clust = Clustering(k=20, algorithm='complete')
y1 = clust.fit_predict(X1)
y2 = clust.fit_predict(X2)

/home/semen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in subtract
/home/semen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in subtract


In [344]:
y1

array([ 0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,
        0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  2,  0,  0,  0,  0,  0,  1,  0,  3,  0,  4,  0,  5,  0,  6,  7,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  8,  0,  0,  0,  9,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10,  0,  0,
        0,  8,  8,  0,  0,  0,  0, 11, 12, 13,  0,  0,  0,  0,  0,  0, 14,
        0,  0,  0,  0,  0, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0, 16,  0,  6,  0,  0,  0,  0,  8,  0,  0,  0,  0,  0,  0,  7,  0,
        0,  0,  0,  0,  0,  0,  0, 17,  8,  0,  0, 10,  0,  0,  0,  0,  0,
        0,  0,  7,  0,  1,  0,  0, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  2,  0,  0,  7,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,
        0,  0,  0,  0,  0

In [345]:
y2

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        1,  2,  0,  0,  0,  0,  0,  0,  0,  3,  4,  5,  0,  2,  0,  2,  2,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  6,  0,  0,  0,  0,  0,  0,  0,
        2,  0,  0,  0,  0,  0,  0,  7,  8,  9, 10,  0,  0,  0,  0,  0, 11,
        0,  2,  0,  0,  0, 12,  0,  0,  0,  7,  0,  0,  0,  0,  0,  0, 13,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,
        0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0, 14,  0,
        0,  0,  0,  0,  0,  0,  0, 14,  0,  0,  0,  0,  0,  0,  0,  2,  0,
        0,  0, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 16,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0

In [346]:
for k in range(10):
    print(df1[y1 == k].index)

Index(['Afghanistan', 'Albania', 'American Samoa', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Arab World', 'Argentina', 'Armenia', 'Aruba',
       ...
       'Uruguay', 'Uzbekistan', 'Vanuatu', 'Venezuela, RB', 'Vietnam',
       'Virgin Islands (U.S.)', 'West Bank and Gaza', 'Yemen, Rep.', 'Zambia',
       'Zimbabwe'],
      dtype='object', name='Country Name', length=226)
Index(['Algeria', 'Benin', 'Chad', 'Philippines', 'South Africa', 'Sweden'], dtype='object', name='Country Name')
Index(['Bangladesh', 'Cameroon', 'Senegal'], dtype='object', name='Country Name')
Index(['Chile'], dtype='object', name='Country Name')
Index(['Colombia'], dtype='object', name='Country Name')
Index(['Congo, Dem. Rep.'], dtype='object', name='Country Name')
Index(['Costa Rica', 'Mexico'], dtype='object', name='Country Name')
Index(['Cote d'Ivoire', 'Myanmar', 'Paraguay', 'Sierra Leone'], dtype='object', name='Country Name')
Index(['Euro area', 'IBRD only', 'IDA & IBRD total', 'Low & middle income

In [347]:
for k in range(10):
    print(df2[y2 == k].index)

Index(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra',
       'Angola', 'Antigua and Barbuda', 'Arab World', 'Argentina', 'Armenia',
       ...
       'United States', 'Upper middle income', 'Uruguay', 'Vanuatu',
       'Venezuela, RB', 'Virgin Islands (U.S.)', 'West Bank and Gaza',
       'Yemen, Rep.', 'Zambia', 'Zimbabwe'],
      dtype='object', name='Country Name', length=230)
Index(['Cambodia', 'Russian Federation', 'Uganda'], dtype='object', name='Country Name')
Index(['Cameroon', 'Congo, Dem. Rep.', 'Costa Rica', 'Cote d'Ivoire',
       'Hungary', 'Kazakhstan', 'Madagascar', 'Mexico', 'Mongolia',
       'Pakistan'],
      dtype='object', name='Country Name')
Index(['Chile'], dtype='object', name='Country Name')
Index(['China'], dtype='object', name='Country Name')
Index(['Colombia'], dtype='object', name='Country Name')
Index(['Guinea'], dtype='object', name='Country Name')
Index(['India', 'Lao PDR'], dtype='object', name='Country Name')
Index(['Indonesia'], dt

Честно говоря, ничего не понял. Получается разреженный предикт, то есть все сваливается в один кластер. Может, оно и правильно, так как я заполнил пропуски нулями, а их было очень много (потому что я очень грубо и бездумно сформировал датасет). Сказать нечего, кроме того, что алгоритм не падает и дает нужное число кластеров.

In [8]:
# Не переживайте, мне тоже было тяжело разобраться с данными

# Контрольные вопросы
Постарайтесь максимально развернуто и честно ответить на вопросы. Они охватывают тему алгоритмов кластеризации и скорее нужны преподавателям, чтобы понимать, что именно Вы усвоили плохо. Надеюсь, они подскажут, что именно в теме Вы не понимаете или наоборот порадают, что Вы все знаете ^_^

* Опишите, пожалуйста, для каких прикладных задач Вы бы стали использовать методы кластеризации?

***Ответ***

Для того, чтобы изучить структуру данных. В проекте я хотел использовать следующий подход: кластеризовать текстовые документы (в виде tf-idf) и в качестве признаков использовать метки кластера, а мешки слов выкинуть. Не получилось, но подход имеет право на существование.

* Какие преимущества и недостатки Вы видите у следуюших алгоритмов кластеризации: kmeans, dbscan, агломеративная кластеризация?

***Ответ***

K-means: быстрый, простой, всегда сходится, но кластеры имеют сферическую форму

dbscan: работает для любой формы, сам определяет число кластеров, но не справляется с кластерами разной плотности

агломеративная кластеризация: может использовать специальные метрики, любое число кластеров "из коробки", но требовательна к ресурсам

* На лекции было показано, что EM алгоритм находит оценку максимального правдоподобия в задаче со сккрытыми переменными. Из курса мат. статистики Вы можете знать, оценка максимального правдоподобия "оптимальная" во многих смыслах. В каких же случаях тогда не следует применять метод максимального правдоподобия?

***Ответ***

Делаются серьезные допущения относительно распределения исследуемых величин. Если не знаем структуру распределения, может не стоит использовать EM?

* Многие алгоритмы анализа данных страдают "проклятием" размерности. Страдают ли этим проклятием алгоритмы кластеризации? Если да, то как бы Вы с этим боролись?

***Ответ***

Страдают. Все изученные нами алгоритмы кластеризации - метрические, а вычисление расстояний в пространствах больших размерностей - дорого. 

* Представьте, что у Вас есть набор текстовых документов, и Вы хотите разбить эти документы на тематики - новости, спорт, кулинария, кино и так далее. Размеченной выборки у Вас нет, только сами тексты документов. Какие алгоритмы и какие признаки Вы бы использовали для решения данной задачи?

***Ответ***

На мой взгляд тут важнее именно признаки, а не метод кластеризации. Если темы такие, что их разумно определять по словам, а не по смыслу, то можно преобразовать тексты tf-idf и использовать иерархическую кластеризацию с текстовыми нормами (ужасающая размерность). Можно попробовать самому выделить ключевые слова, one-hot-encoding, вхождения этих слов использовать как признаки. Или не самому, а с помощью какого-нибудь статистического критерия.

* Теперь представьте, что небольшая часть Ваших текстов была размечена людьми, но большая часть так и осталась без таргета. Смогли ли бы Вы использовать эти данные для улучения качества кластеризации? (эта задача называется semi-supervised learning)

***Ответ***

Использовать для валидации модели. 

На мой взгляд, любая дополнительная априорная информация это полезно, но как именно ее применять я не вижу.

Здесь Вы можете оставить отзыв о этой домашней работе или о всем курсе.   

*** ВАШ ОТЗЫВ ЗДЕСЬ***

Отличный курс, все круто! Спасибо!